In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
id_gpu = '2'
n_users = 43
os.environ['CUDA_VISIBLE_DEVICES'] = id_gpu

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
num_ts = 862

In [6]:
# Download the dataset
download_traffic_dataset()

# Clean the dataset
traffic_data = clean_traffic(num_ts=num_ts)
traffic_data.shape

Files not exist
/home/qula0496/quan/Nonstationary_Transformers/utils/data_utils.py
data loaded..


(17544, 862)

In [7]:
traffic_data.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_853,MT_854,MT_855,MT_856,MT_857,MT_858,MT_859,MT_860,MT_861,MT_862
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,0.0048,0.0146,0.0289,0.0142,0.0064,0.0232,0.0162,0.0242,0.0341,0.0375,...,0.0051,0.0051,0.0074,0.0079,0.0051,0.0051,0.0339,0.0051,0.0100,0.0121
2015-01-01 01:00:00,0.0072,0.0148,0.0350,0.0174,0.0084,0.0240,0.0201,0.0338,0.0434,0.0381,...,0.0036,0.0036,0.0107,0.0058,0.0036,0.0036,0.0348,0.0036,0.0087,0.0136
2015-01-01 02:00:00,0.0040,0.0101,0.0267,0.0124,0.0049,0.0170,0.0127,0.0255,0.0332,0.0309,...,0.0030,0.0030,0.0043,0.0050,0.0030,0.0030,0.0327,0.0030,0.0061,0.0107
2015-01-01 03:00:00,0.0039,0.0060,0.0218,0.0090,0.0029,0.0118,0.0088,0.0163,0.0211,0.0199,...,0.0033,0.0033,0.0019,0.0052,0.0033,0.0033,0.0292,0.0033,0.0040,0.0071
2015-01-01 04:00:00,0.0042,0.0055,0.0191,0.0082,0.0024,0.0095,0.0064,0.0087,0.0144,0.0226,...,0.0049,0.0049,0.0011,0.0071,0.0049,0.0049,0.0264,0.0049,0.0040,0.0039


In [8]:
# # prompt: write pandas to csv

# import pandas as pd

# # Save the DataFrame to a CSV file
# traffic_data.to_csv('/home/qula0496/quan/Nonstationary_Transformers/dataset/traffic/traffic.csv', index_label='date')

In [9]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(traffic_data.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = id_gpu
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.root_path = './dataset/traffic/'
args.data_path = 'traffic.csv'
args.model_id = 'traffic_96_96'
args.data_path

'traffic.csv'

In [10]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [11]:
test_data

In [12]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, traffic_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, traffic_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [13]:
from datetime import datetime
datetime.strptime('2015-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')

datetime.datetime(2015, 1, 1, 0, 0)

In [14]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(20, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(20, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
          

In [15]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 30
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.1
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Traffic dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Traffic dataset",
    # Hyperparams
    config={
        "dataset": "Traffic860",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 860,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [16]:
# len(server.test_loader[0])

In [17]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 cost time: 128.77880334854126
Epoch: 2 cost time: 149.21691060066223
Epoch: 3 cost time: 223.0167372226715
Epoch: 4 cost time: 245.58553194999695
Epoch: 5 cost time: 264.81232023239136
Epoch: 1 cost time: 265.80989956855774
Epoch: 2 cost time: 266.061963558197
Epoch: 3 cost time: 266.995322227478
Epoch: 4 cost time: 268.28659105300903
Epoch: 5 cost time: 266.11939430236816
Epoch: 1 cost time: 265.2906529903412
Epoch: 2 cost time: 265.53083181381226
Epoch: 3 cost time: 266.8143925666809
Epoch: 4 cost time: 266.62430596351624
Epoch: 5 cost time: 266.4717514514923
Epoch: 1 cost time: 266.7112572193146
Epoch: 2 cost time: 264.87778520584106
Epoch: 3 cost time: 266.1696357727051
Epoch: 4 cost time: 267.3686966896057
Epoch: 5 cost time: 267.8868942260742


Progress:   3%|▎         | 1/30 [1:30:22<43:40:44, 5422.23s/it]

Epoch: 1 cost time: 266.6789638996124
Epoch: 2 cost time: 267.5209991931915
Epoch: 3 cost time: 267.1515202522278
Epoch: 4 cost time: 266.4000873565674
Epoch: 5 cost time: 267.16089820861816
Epoch: 1 cost time: 266.8360993862152
Epoch: 2 cost time: 267.30051445961
Epoch: 3 cost time: 266.4577851295471
Epoch: 4 cost time: 266.45491886138916
Epoch: 5 cost time: 268.2099003791809
Epoch: 1 cost time: 264.7796673774719
Epoch: 2 cost time: 266.4899032115936
Epoch: 3 cost time: 266.72820687294006
Epoch: 4 cost time: 266.99161863327026
Epoch: 5 cost time: 265.53781056404114
Epoch: 1 cost time: 268.7816138267517
Epoch: 2 cost time: 264.7610638141632
Epoch: 3 cost time: 264.74677658081055
Epoch: 4 cost time: 265.65533995628357
Epoch: 5 cost time: 267.9452784061432


Progress:   7%|▋         | 2/30 [3:06:10<43:39:53, 5614.04s/it]

...

Progress:  93%|█████████▎| 28/30 [32:15:04<1:52:41, 3380.80s/it]

Epoch: 1 cost time: 142.39138555526733
Epoch: 2 cost time: 142.3800048828125
Epoch: 3 cost time: 143.04570817947388
Epoch: 4 cost time: 142.63516640663147
Epoch: 5 cost time: 141.59208631515503
Epoch: 1 cost time: 143.05583262443542
Epoch: 2 cost time: 141.88444995880127
Epoch: 3 cost time: 142.14557600021362
Epoch: 4 cost time: 143.45768904685974
Epoch: 5 cost time: 140.9634268283844
Epoch: 1 cost time: 141.3039882183075
Epoch: 2 cost time: 143.68184304237366
Epoch: 3 cost time: 142.96706104278564
Epoch: 4 cost time: 143.66659545898438
Epoch: 5 cost time: 144.144268989563
Epoch: 1 cost time: 144.62233638763428
Epoch: 2 cost time: 119.11511754989624
Epoch: 3 cost time: 116.17452025413513
Epoch: 4 cost time: 116.26573753356934
Epoch: 5 cost time: 120.64423847198486


Progress:  97%|█████████▋| 29/30 [33:04:08<54:09, 3249.65s/it]  

Epoch: 1 cost time: 113.76352596282959
Epoch: 2 cost time: 111.37110757827759
Epoch: 3 cost time: 113.2837336063385
Epoch: 4 cost time: 111.24582362174988
Epoch: 5 cost time: 112.99351787567139
Epoch: 1 cost time: 112.21153211593628
Epoch: 2 cost time: 112.0816388130188
Epoch: 3 cost time: 111.50443816184998
Epoch: 4 cost time: 112.05832815170288
Epoch: 5 cost time: 112.19026708602905
Epoch: 1 cost time: 111.82082962989807
Epoch: 2 cost time: 113.30190992355347
Epoch: 3 cost time: 112.46572589874268
Epoch: 4 cost time: 112.50230550765991
Epoch: 5 cost time: 109.82794761657715
Epoch: 1 cost time: 111.70939230918884
Epoch: 2 cost time: 112.01958274841309
Epoch: 3 cost time: 113.5833191871643
Epoch: 4 cost time: 113.56173419952393
Epoch: 5 cost time: 113.66753959655762


Progress: 100%|██████████| 30/30 [33:44:31<00:00, 4049.05s/it]


In [18]:
total_mae

[0.26244587,
 0.29568663,
 0.23205206,
 0.25478095,
 0.28187844,
 0.37863362,
 0.31492305,
 0.30888048,
 0.34229752,
 0.26122627,
 0.32674596,
 0.2626313,
 0.26402968,
 0.26033714,
 0.3715763,
 0.2646031,
 0.23957962,
 0.2509775,
 0.24376087,
 0.25778008,
 0.2856703,
 0.28105497,
 0.30834326,
 0.29272947,
 0.3616104,
 0.38590533,
 0.35685366,
 0.2399795,
 0.32879335,
 0.2807287,
 0.32140052,
 0.29822317,
 0.30820486,
 0.37697136,
 0.36903372,
 0.34757096,
 0.39956367,
 0.32349962,
 0.32898465,
 0.3685847,
 0.32926857,
 0.35520858,
 0.31802034]